In [84]:
import csv
import glob
import os
import random
import time
import requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
# ------------------------------------------------------------------------------
# [설정] 헤더 및 URL
# ------------------------------------------------------------------------------
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0.0"
}
recipe_url = "https://www.10000recipe.com/recipe/list.html?order=reco&cat4=56&page="
base_url = "https://www.10000recipe.com/recipe/"

fieldnames = [
    'Recipe_Link', 'Title', 'Serving', 'Preparation_Time', 'Difficulty',
    'Main_Thumbnail', 'Summary_In', 'Ingredient', 'Condiment', 'Steps'
]

# ------------------------------------------------------------------------------
# [1] 이어하기(Resume) 로직 설정
# ------------------------------------------------------------------------------
print("=" * 60)
print(" [만개의 레시피 크롤러] 시작 설정")
print("=" * 60)

# 기존 결과 파일 찾기 (최신순 정렬)
existing_files = sorted(glob.glob("10000recipe_result_*.csv"), key=os.path.getmtime, reverse=True)

filename = ""
processed_links = set()  # 이미 수집 완료된 레시피 링크 저장소

if existing_files:
    latest_file = existing_files[0]
    print(f"\n[알림] 최근 작업 파일을 발견했습니다: '{latest_file}'")
    resume = input("  -> 이 파일에 이어서 작업하시겠습니까? (y/n): ").strip().lower()

    if resume == 'y':
        filename = latest_file
        print(f"  -> '{filename}' 파일을 분석 중입니다...")
        try:
            with open(filename, 'r', encoding='utf-8-sig') as f:
                reader = csv.DictReader(f)
                for row in reader:
                    if 'Recipe_Link' in row and row['Recipe_Link']:
                        processed_links.add(row['Recipe_Link'])
            print(f"  -> ✅ 복구 완료: 총 {len(processed_links)}개의 레시피가 이미 수집되어 있습니다. (건너뜀)")
        except Exception as e:
            print(f"  -> [오류] 파일 읽기 실패. 새 파일로 시작합니다. ({e})")
            filename = ""
    else:
        print("  -> 새 파일로 시작합니다.")

# 파일명이 없으면(새로 시작) 생성
if not filename:
    filename = f"10000recipe_result_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    # 새 파일이면 헤더 미리 작성
    with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
    print(f"\n📁 새로운 파일 생성: '{filename}'")

print(f"📁 모든 데이터는 '{filename}' 파일에 실시간 저장됩니다.\n")
time.sleep(2)

# ------------------------------------------------------------------------------
# [2] 크롤링 시작
# ------------------------------------------------------------------------------
food_data = []
count_success = 0  # 금일 성공 횟수

try:
    # --- 1. 음식 목록 긁어 오기 (목록은 빠르게 훑어야 하므로 전체 스캔) ---
    # (이미 수집한 ID라도 '제목/이미지' 등의 정보가 목록에 있으므로 목록 수집은 수행)

    # 테스트를 위해 범위를 500으로 유지합니다. (필요시 조절하세요)
    # 목록 수집 단계는 비교적 빠르므로 전체를 돕니다.
    print(">>> [단계 1/2] 페이지 목록 수집 시작 (총 500페이지)...")

    for i in tqdm(range(500), desc="목록 수집 중"):
        page_no = i + 1
        try:
            response = requests.get(recipe_url + str(page_no), headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            food_items = soup.select('li.common_sp_list_li')

            for item in food_items:
                id_tag = item.select_one('a.common_sp_link')
                title_tag = item.select_one('div.common_sp_caption_tit')

                # 안전한 추출
                id_val = id_tag['href'].replace('/recipe/', '') if id_tag and 'href' in id_tag.attrs else None
                title = title_tag.get_text(strip=True) if title_tag else 'None'

                # 필요한 정보만 임시 저장 (상세 페이지에서 추가 수집)
                if id_val:
                    food_data.append({
                        'ID': id_val,
                        'Title': title,
                        'Page': page_no
                    })

            time.sleep(random.uniform(0.3, 0.6)) # 목록 수집은 조금 빠르게

        except Exception as e:
            print(f"[Error] 목록 {page_no}페이지 수집 중 오류: {e}")
            continue

    print(f"\n✅ 목록 수집 완료. 총 {len(food_data)}개의 후보 레시피 확보.")
    print(">>> [단계 2/2] 상세 정보 수집 및 필터링 시작...")
    time.sleep(1)

    # --- 2. 레시피 상세 정보 긁어 오기 ---
    total_items = len(food_data)

    # 진행률 표시바 생성
    pbar = tqdm(food_data, desc="상세 수집 중")

    for idx, food in enumerate(pbar, start=1):
        try:
            recipe_link = base_url + str(food['ID'])

            # [핵심] 이미 수집된 링크면 건너뛰기
            if recipe_link in processed_links:
                # 너무 로그가 빠르면 보기 힘드므로 tqdm에는 표시 안 하거나 간략히 처리
                continue

            # pbar 설명 업데이트
            pbar.set_postfix({"Pg": food['Page'], "Add": count_success})

            # 상세 페이지 요청
            response = requests.get(recipe_link, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # --------------------------------
            # 조건 필터링 (1인분 / 초급 등)
            # --------------------------------
            serv_tag = soup.select_one('span.view2_summary_info1')
            prep_tag = soup.select_one('span.view2_summary_info2')
            diff_tag = soup.select_one('span.view2_summary_info3')

            difficulty = diff_tag.get_text(strip=True) if diff_tag else None
            # 조건: '아무나' 또는 '초급'이 아니면 건너뜀
            if difficulty not in ['아무나', '초급']:
                continue

            serving = serv_tag.get_text(strip=True) if serv_tag else None
            # 조건: '1인분'이 아니면 건너뜀
            if not serving or serving != '1인분':
                continue

            prep_time = prep_tag.get_text(strip=True) if prep_tag else None
            if not prep_time:
                continue

            # --------------------------------
            # 데이터 추출
            # --------------------------------
            main_thumb_id = soup.select_one('#main_thumbs')
            main_thumb = main_thumb_id['src'] if main_thumb_id else 'None'

            summary_in_id = soup.select_one('#recipeIntro')
            summary_in = summary_in_id.get_text(strip=True) if summary_in_id else 'None'

            # 재료 추출
            ingred_items = soup.select('div.ready_ingre3 ul')
            ingredient_text = 'None'
            condiment_text = 'None'

            if not ingred_items:
                continue
            else:
                for item in ingred_items:
                    title_tag = item.select_one('b.ready_ingre3_tt')
                    group_title = title_tag.get_text(strip=True) if title_tag else 'None'
                    ingredients = item.select('li')
                    ingredient_list = []
                    for ingredient_item in ingredients:
                        name_tag = ingredient_item.select_one('div.ingre_list_name a') or ingredient_item.select_one('div.ingre_list_name')
                        ea_tag = ingredient_item.select_one('span.ingre_list_ea')
                        i_name = name_tag.get_text(strip=True) if name_tag else ''
                        i_ea = ea_tag.get_text(strip=True) if ea_tag else ''
                        if i_name:
                            ingredient_list.append(f'{i_name}-{i_ea}')

                    joined_ingredients = ','.join(ingredient_list)
                    if '재료' in group_title or '레시피' in group_title:
                        ingredient_text = joined_ingredients
                    elif '양념' in group_title:
                        condiment_text = joined_ingredients

            # 조리 순서 추출
            step_divs = soup.select('div.view_step_cont')
            step_list = []
            skip_recipe = False

            for i, step_div in enumerate(step_divs):
                desc_tag = step_div.select_one('div.media-body')
                desc = desc_tag.get_text(strip=True) if desc_tag else "None"

                img_tag = step_div.select_one('img[src*="/recipe/"]')
                if not img_tag:
                    img_tag = step_div.select_one('img')
                img_src = img_tag['src'] if img_tag else "None"

                # 이미지가 없으면 레시피 제외 (사용자 로직 유지)
                if img_src == 'None':
                    skip_recipe = True
                    break

                step_str = f"{i + 1}단계||{desc}||{img_src}"
                step_list.append(step_str)

            if skip_recipe:
                continue

            all_steps_text = "\n".join(step_list)

            # --------------------------------
            # [저장] 실시간 CSV 저장 (Append)
            # --------------------------------
            row_data = {
                'Recipe_Link': recipe_link,
                'Title': food['Title'],
                'Serving': serving,
                'Preparation_Time': prep_time,
                'Difficulty': difficulty,
                'Main_Thumbnail': main_thumb,
                'Summary_In': summary_in,
                'Ingredient': ingredient_text,
                'Condiment': condiment_text,
                'Steps': all_steps_text
            }

            # 파일 끝에 추가 (Header 없이)
            with open(filename, 'a', newline='', encoding='utf-8-sig') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow(row_data)

            count_success += 1
            processed_links.add(recipe_link) # 메모리에도 추가 (혹시 모를 중복 방지)

            # 속도 조절
            time.sleep(random.uniform(0.5, 1.0))

        except KeyboardInterrupt:
            # 루프 중간에 인터럽트 발생 시 바로 밖으로 던짐
            raise KeyboardInterrupt
        except Exception as e:
            # 개별 에러는 무시하고 다음 레시피로
            print(f"  [Skip] ID {food.get('ID')} 에러: {e}")
            continue

except KeyboardInterrupt:
    print("\n\n⛔ [사용자 중지] 작업을 멈춥니다.")

finally:
    print(f"\n🎉 작업 종료! 금일 {count_success}개의 레시피가 '{filename}'에 추가되었습니다.")